# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [3]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
import json
import csv

# Import API key
from api_keys import g_key

gmaps.configure(g_key)



### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [4]:
 #import csv as data frame


city_data_test = pd.read_csv(".\output\city_data.csv")
#city_data_test.head(10)

#convert strings to floats in data frame

# city_data_test["Humidity"] = city_data_test["Humidity"].astype(float)
city_data_test["Humidity"] = city_data_test["Humidity"].astype(float)
city_data_test["Wind Speed"] = city_data_test["Wind Speed"].astype(float)
city_data_test["Max Temp"] = city_data_test["Max Temp"].astype(float)
city_data_test["Cloudiness"] = city_data_test["Cloudiness"].astype(float)
#city_data_test["Date"] = pd.to_datetime(city_data_test["Date"], format = '%Y%m%d')


#Drop cities that are missing value & shows stats for test
clean_city_data_test = city_data_test.dropna()

#clean_city_data_test.describe()

clean_city_data_test.head()


,City,Lat,Lng,Max Temp,Humidity,Wind Speed,Cloudiness,Country,Date
0,Port Hedland,59.70,30.79,45.00,93.0,4.47,90.0,AU,1.604115e+09
1,New Norfolk,33.28,134.15,68.00,30.0,3.36,20.0,AU,1.604115e+09
3,Taolanaro,5.89,95.32,82.31,74.0,10.65,98.0,MG,1.604115e+09
4,Mount Gambier,3.07,172.79,82.58,76.0,19.42,63.0,AU,1.604114e+09
5,Mahebourg,63.29,18.72,33.80,85.0,2.24,100.0,MU,1.604115e+09


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [5]:
# Create city humidity heat Map of all of Part 1s data 

# Configure gmaps
gmaps.configure(api_key = g_key)

# Store latitude and longitude in locations
locations = clean_city_data_test[["Lat", "Lng"]]

# Convert Humidity to float JIC
rating = clean_city_data_test["Humidity"] 

# Plot Heatmap
fig_13 = gmaps.figure()

# Create heat layer
heat_layer = gmaps.heatmap_layer(locations, weights=rating, 
                                 dissipating=False, max_intensity=100,
                                 point_radius=2)

# Add layer
fig_13.add_layer(heat_layer)

# Display figure
fig_13

# Save a png
#fig_13.figure.savefig('.\output\City_Humidity Heatmap.png')

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [6]:
clean_city_data_test.head()
# Reduce amount based on Weather criteria  Humidity <= 75.  Max Temp > 50.  Wind Speed <10.  Cloudiness <30

ideal_city = clean_city_data_test.loc[(clean_city_data_test["Wind Speed"] <25) & 
                                      (clean_city_data_test["Humidity"] < 90) &
                                      (clean_city_data_test["Max Temp"] > 2) &
                                      (clean_city_data_test["Wind Speed"] < 10) &
                                      (clean_city_data_test["Cloudiness"] < 30)&
                                      (clean_city_data_test["Lat"] >40 )&
                                      (clean_city_data_test["Lng"] < 50 )&
                                      (clean_city_data_test["Lng"] > -10 )                                      
                                     ]


ideal_city.head(7)
ideal_city.to_csv(r'.\output\ideal_city.csv')


In [7]:
# search for hotels & create new df

#import ideal cities
hotel_df = pd.read_csv('.\output\ideal_city.csv')
hotel_df = hotel_df.drop('Date', 1)


# Add Hotel Name 
hotel_df['Hotel Name'] = ""
del hotel_df['Unnamed: 0']
hotel_df.head()


,City,Lat,Lng,Max Temp,Humidity,Wind Speed,Cloudiness,Country,Hotel Name
0,Bredasdorp,78.22,15.64,17.60,72.0,1.12,20.0,ZA,
1,Port Elizabeth,56.00,8.13,46.99,75.0,9.66,16.0,ZA,
2,Bredasdorp,78.22,15.64,17.60,72.0,1.12,20.0,ZA,
3,Busselton,78.22,15.64,17.60,72.0,1.12,20.0,AU,
4,Kirakira,78.22,15.64,17.60,72.0,1.12,20.0,SB,


In [8]:
# find closest hotels

# Define key search parameters
target_keyword = "hotel"
target_radius = 5000
target_type = "hotel"

base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

for index, row in hotel_df.iterrows():
    lat = row[1]
    lng = row[2]
    lat_lon =f"{lat},{lng}"
    city = row[0]
    
    params = {
        "location": lat_lon,
        "keyword": target_keyword,
        "radius": target_radius,
        "type": target_type,
        "key": g_key
    }   

    response = requests.get(base_url, params=params).json()    
#     print(json.dumps(response, indent=4, sort_keys=True))    
  
# Cannot figure out how to get past this point.  I keep receiving a TypeError: list indices must be integers or slices, not str
# I Discovered that my issue was the I was missing the [0] JSON in the response syntax
# Before: hotel_name = response["results"]["name"] (incorrect)
# AFTER: hotel_name = response["results"][0]["name"] 

    try:
        hotel_name = response["results"][0]["name"]      
        hotel_df.loc[index, 'Hotel Name'] = hotel_name
        print(f'{row[0]} is complete with hotel: {hotel_name}')
    except Exception:
        print("Missing field")
        hotel_df.loc[index, 'Hotel Name'] = "No hotel"
    print('--------')    


Bredasdorp is complete with hotel: Radisson Blu Polar Hotel, Spitsbergen
--------
Port Elizabeth is complete with hotel: Drivethru Surfcamps
--------
Bredasdorp is complete with hotel: Radisson Blu Polar Hotel, Spitsbergen
--------
Busselton is complete with hotel: Radisson Blu Polar Hotel, Spitsbergen
--------
Kirakira is complete with hotel: Radisson Blu Polar Hotel, Spitsbergen
--------
Albany is complete with hotel: Radisson Blu Polar Hotel, Spitsbergen
--------
Grand River South East is complete with hotel: Vardø Hotel AS
--------
Mahebourg is complete with hotel: Hotel Alma
--------
Albany is complete with hotel: Vardø Hotel AS
--------
Saint-Joseph is complete with hotel: Radisson Blu Polar Hotel, Spitsbergen
--------
Obluche is complete with hotel: Panorama Hotel
--------


In [10]:
hotel_df.head(15)
# print(hotel_df[['Lat', 'Lng']])

,City,Lat,Lng,Max Temp,Humidity,Wind Speed,Cloudiness,Country,Hotel Name
0,Bredasdorp,78.22,15.64,17.60,72.0,1.12,20.0,ZA,"Radisson Blu Polar Hotel, Spitsbergen"
1,Port Elizabeth,56.00,8.13,46.99,75.0,9.66,16.0,ZA,Drivethru Surfcamps
2,Bredasdorp,78.22,15.64,17.60,72.0,1.12,20.0,ZA,"Radisson Blu Polar Hotel, Spitsbergen"
3,Busselton,78.22,15.64,17.60,72.0,1.12,20.0,AU,"Radisson Blu Polar Hotel, Spitsbergen"
4,Kirakira,78.22,15.64,17.60,72.0,1.12,20.0,SB,"Radisson Blu Polar Hotel, Spitsbergen"
5,Albany,78.22,15.64,17.60,72.0,1.12,20.0,AU,"Radisson Blu Polar Hotel, Spitsbergen"
6,Grand River South East,70.37,31.11,34.03,87.0,6.80,27.0,MU,Vardø Hotel AS
7,Mahebourg,62.27,21.38,42.17,87.0,5.14,1.0,MU,Hotel Alma
8,Albany,70.37,31.11,34.03,87.0,6.80,27.0,AU,Vardø Hotel AS
9,Saint-Joseph,78.22,15.64,17.60,72.0,1.12,20.0,RE,"Radisson Blu Polar Hotel, Spitsbergen"


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [11]:
# Add marker layer ontop of heat map & Display figure

# Convert Hotel Name to List
hotel_name = hotel_df["Hotel Name"].tolist()

# Generate Marker Map Layer
marker_locations = hotel_df[['Lat', 'Lng']]

fig_14 = gmaps.figure()
markers = gmaps.marker_layer(marker_locations, 
                            info_box_content=[f"{hotel}" for hotel in hotel_name])
                            
fig_13.add_layer(markers)
fig_13

Figure(layout=FigureLayout(height='420px'))

In [ ]:
# How to Create Markers on a Gmap (Census_States)
# # Create a map using state centroid coordinates to set markers
# marker_locations = census_data[['Latitude', 'Longitude']]

# # Create a marker_layer using the poverty list to fill the info box
# fig = gmaps.figure()
# markers = gmaps.marker_layer(marker_locations,
#     info_box_content=[f"Poverty Rate: {rate}" for rate in poverty_rate])
# fig.add_layer(markers)
# fig

In [ ]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
# info_box_template = """
# <dl>
# <dt>Name</dt><dd>{Hotel Name}</dd>
# <dt>City</dt><dd>{City}</dd>
# <dt>Country</dt><dd>{Country}</dd>
# </dl>
# """
# # Store the DataFrame Row
# # NOTE: be sure to update with your DataFrame name
# hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
# locations = hotel_df[["Lat", "Lng"]]

In [ ]:
# Add marker layer ontop of heat map


# Display figure
